# Use to Check Reproducibility - v2.2 Code

Version 2.1 relies on conda environment built from yaml file `fmda/install/fmda_ml.yml`. This environment uses python version `>=3.12` tensorflow version `>=2.16`. This led to substantial changes from the old reproducibility for code v2.0, so those old reproducibility hashes are no longer supported.

To see old code and reproduciblity results, see commits tagged with "REPRODUCIBILITY RESET".

## Setup

In [ ]:
# Environment
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import sys
# Local modules
sys.path.append('..')
import reproducibility
from utils import print_dict_summary, read_yml, read_pkl
from moisture_rnn_pkl import pkl2train
from moisture_rnn import RNN, RNNData, RNNParams
import logging
from utils import logging_setup
logging_setup()

### Reproducibility Dataset

In [ ]:
# Original File
repro_file='../data/reproducibility_dict_v2.2.pkl'
repro = read_pkl(repro_file)

## RNN Single Case

In [ ]:
# Set up params
params = repro['single']['repro_info']['params']
print(type(params))
print(params)

In [ ]:
# Set up input data
rnn_dat = RNNData(repro['single'], scaler = params['scaler'], features_list = params['features_list'])
rnn_dat.train_test_split(
    time_fracs = params['time_fracs']
)
rnn_dat.scale_data()
rnn_dat.batch_reshape(timesteps = params['timesteps'], batch_size = params['batch_size'])

In [ ]:
reproducibility.set_seed()
rnn = RNN(params)
m, errs = rnn.run_model(rnn_dat, reproducibility_run=True, plot_period="predict")

### Physics Initialized

In [ ]:
params.update({
    'phys_initialize': True,
    'scaler': None,
    'features_list': ['Ed', 'Ew', 'rain'],
    'dense_layers': 0
})

In [ ]:
# Set up input data
rnn_dat = RNNData(repro['single'], scaler = params['scaler'], features_list = params['features_list'])
rnn_dat.train_test_split(
    time_fracs = params['time_fracs']
)
rnn_dat.batch_reshape(timesteps = params['timesteps'], batch_size = params['batch_size'])

In [ ]:
reproducibility.set_seed()
rnn = RNN(params)
m, errs = rnn.run_model(rnn_dat, reproducibility_run=True, plot_period='predict')

## Spatial Case

In [ ]:
# Set up params
params = repro['spatial']['repro_info']['params']
print(type(params))
print(params)

In [ ]:
repro['single']['y'].shape[0]

In [ ]:
# Set up input data
rnn_dat = RNNData(repro['spatial'], scaler = params['scaler'], features_list = params['features_list'])
rnn_dat.train_test_split(
    time_fracs = [.5, .2, .3],
    space_fracs = params['space_fracs']
)
rnn_dat.scale_data()
rnn_dat.batch_reshape(timesteps = params['timesteps'], batch_size = params['batch_size'])

In [ ]:
params.update({
    'loc_batch_reset': rnn_dat.n_seqs # Used to reset hidden state when location changes for a given batch
})

In [ ]:
import importlib
import moisture_rnn
importlib.reload(moisture_rnn)
from moisture_rnn import RNN

In [ ]:
reproducibility.set_seed()
rnn = RNN(params)
m, errs = rnn.run_model(rnn_dat, reproducibility_run=True)

In [ ]:
errs.mean()